# 0. Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/industry_classification

/content/drive/MyDrive/industry_classification


In [ ]:
!pip install transformers -qq
!pip install datasets -qq
!pip install wandb -qq
!pip install scikit-learn -qq

In [ ]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm, tqdm_notebook
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from matplotlib import pyplot as plt
import seaborn as sns

from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
from transformers import TrainingArguments, Trainer

# 1. Prepare the Data

### 1-1. Take text chunk from class table by subcategories

In [ ]:
df_eng = pd.read_excel('data/한국표준산업분류(10차)_영문.xlsx')

In [ ]:
df_eng

,"A 농업, 임업 및 어업(01~03)",Unnamed: 1,Unnamed: 2
0,"Agriculture, forestry and fishing",NaN,NaN
1,01,농업,Agriculture
2,011,작물 재배업,Growing of crops
3,0111,곡물 및 기타 식량작물 재배업,Growing of cereal crops and other crops for food
4,01110,곡물 및 기타 식량작물 재배업,Growing of cereal crops and other crops for food
...,...,...,...
2068,99,국제 및 외국기관,Activities of extraterritorial organizations a...
2069,990,국제 및 외국기관,Activities of extraterritorial organizations a...
2070,9900,국제 및 외국기관,Activities of extraterritorial organizations a...
2071,99001,주한 외국공관,Foreign officail residence


In [ ]:
df_eng = df_eng.rename(columns={'A 농업, 임업 및 어업(01~03)':'code', 'Unnamed: 1':'korean_name', 'Unnamed: 2':'english_name'})

In [ ]:
df_eng = df_eng.dropna(how='any')

In [ ]:
low_dict = {}
low_dict_eng = {}
tmp = []
tmp_eng = []
for code, kor_txt, eng_txt in zip(df_eng.loc[:,'code'], df_eng.loc[:,'korean_name'], df_eng.loc[:,'english_name']):
  if len(str(code)) == 2:
    mid_level = kor_txt
    mid_level_eng = eng_txt

  if len(str(code)) == 3:
    tmp = [mid_level]
    tmp.append(kor_txt)
    low_dict[code] = tmp

    tmp_code = code

    tmp_eng = [mid_level_eng]
    tmp_eng.append(eng_txt)
    low_dict_eng[code] = tmp_eng

  if len(str(code)) == 4:
    low_dict[tmp_code].append(kor_txt)
    low_dict_eng[tmp_code].append(eng_txt)

  if len(str(code)) == 5:
    low_dict[tmp_code].append(kor_txt)
    low_dict_eng[tmp_code].append(eng_txt)

In [ ]:
for key, value in low_dict.items():
  low_dict[key] = ' '.join(list(set(value))) # 중복 제거 후 문자열 변환
list(low_dict.items())[:10]

[('011',
  '채소작물 재배업 채소, 화훼작물 및 종묘 재배업 과실작물 재배업 시설작물 재배업 화훼작물 재배업 과실, 음료용 및 향신용 작물 재배업 기타 작물 재배업 곡물 및 기타 식량작물 재배업 농업 기타 시설작물 재배업 음료용 및 향신용 작물 재배업 작물 재배업 종자 및 묘목 생산업 채소, 화훼 및 과실작물 시설 재배업 콩나물 재배업'),
 ('012',
  '양계업 기타 축산업 소 사육업 축산업 양돈업 육우 사육업 가금류 및 조류 사육업 농업 젖소 사육업 기타 가금류 및 조류 사육업 그 외 기타 축산업 말 및 양 사육업'),
 ('013', '작물재배 및 축산 복합농업 농업'),
 ('014',
  '작물재배 관련 서비스업 작물재배 및 축산 관련 서비스업 농업 농산물 건조, 선별 및 기타 수확 후 서비스업 축산 관련 서비스업 작물재배 지원 서비스업'),
 ('015', '수렵 및 관련 서비스업 농업'),
 ('020', '임업용 종묘 생산업 영림업 벌목업 육림업 임업 관련 서비스업 임업 임산물 채취업'),
 ('031', '해수면 어업 원양 어업 연근해 어업 어업 내수면 어업 어로 어업'),
 ('032',
  '해수면 양식 어업 수산물 부화 및 수산종자 생산업 어업 내수면 양식 어업 양식어업 및 어업관련 서비스업 어업 관련 서비스업 양식 어업'),
 ('051', '석탄, 원유 및 천연가스 광업 석탄 광업'),
 ('052', '석탄, 원유 및 천연가스 광업 원유 및 천연가스 채굴업')]

In [ ]:
for key, value in low_dict_eng.items():
  low_dict_eng[key] = ' '.join(list(set(value))) # 중복 제거 후 문자열 변환
list(low_dict_eng.items())[:10]

[('011',
  'Growing of beverage and spice crops Growing of crops Growing of cereal crops and other crops for food Agriculture Growing of vegetable crops, horticultural and nursery products Growing of horticultural products Growing of seed crops and nursery products Growing of vegetable crops Growing of vegetable, fruit crops and horticultural products under cover Growing of bean sprouts Growing of other crops Growing of other crops under cover Growing of fruit Growing of crops under cover  Growing of fruit, beverage and spice crops'),
 ('012',
  'Raising of dairy cattle Raising of pigs Raising of other poultry and birds Raising of poultry and other birds Agriculture Raising of horses, sheep and goats Raising of other animals n.e.c. Raising of cattle Raising of chickens Raising of beef cattle Animal production Raising of other animals'),
 ('013',
  'Agriculture Growing of crops combined with rasinging of animals : mixed farming'),
 ('014',
  'Drying, sorting, grading of agricultural pro

In [ ]:
print(len(low_dict),len(low_dict_eng))

232 232


In [ ]:
pd.DataFrame(list(low_dict.items()), columns = ["code", "chunk_text"]).to_csv("class_table_chunk_kor.csv", index=False)
pd.DataFrame(list(low_dict_eng.items()), columns = ["code", "chunk_text"]).to_csv("class_table_chunk_eng.csv", index=False)

### 1-2. Load Train and Test Data & EDA

In [ ]:
train = pd.read_csv('data/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('data/2. 모델개발용자료.txt', sep='|', encoding='cp949')
submission = pd.read_csv("data/답안 작성용 파일.csv", encoding='cp949')

In [ ]:
train.sample(5)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
895826,id_0895827,P,85,856,복싱학원에서,원생에게,무에타이강습
209867,id_0209868,I,56,561,숙박 및 음식점업,NaN,기관구내식당업
37922,id_0037923,C,25,259,철,밀링가공,반제품
822864,id_0822865,I,56,561,음식점에서,접객시설없이,"삼겹,목살,배달전문"
76458,id_0076459,L,68,682,사업장에서,일반인대상,"전,월세 임대, 매매"


In [ ]:
# 산업분류표 표기는 01~08이지만 digit_2는 1~8(int형 변수)
train[train['digit_2']<10].sample(5)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
63931,id_0063932,B,7,72,해수,염전시설을 갖추고,천일염생산
395496,id_0395497,B,7,72,해수,해수 증발,천일염생산
438830,id_0438831,A,1,11,육묘시설에서,농가를 대상으로,모종판매
981227,id_0981228,B,7,71,원석,"발파, 분쇄",NaN
528362,id_0528363,B,7,71,채석장에서,채굴,토사석채취


In [ ]:
test.sample(5)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
46701,id_046702,NaN,NaN,NaN,사무실에서,화물차 지입주들 대상,업무지원
18221,id_018222,NaN,NaN,NaN,유치원,아동대상,유아교육 서비스
90957,id_090958,NaN,NaN,NaN,사업징에서,고객의뢰,인테리어공사
80727,id_080728,NaN,NaN,NaN,피자전문점에서,고객의 주문에 따라,피자 제공
69704,id_069705,NaN,NaN,NaN,환봉,CNC가공,중장비부품(기어부품)


In [ ]:
submission.sample(5)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
33532,id_033533,NaN,NaN,NaN,음식점에서,접객시설을갖우고,삼겹살구이
98342,id_098343,NaN,NaN,NaN,영업점에서,고객의 주문에 의하여,출장으로 도배 마루 실내장식 서비스
36524,id_036525,NaN,NaN,NaN,지상철에서,일반인대상,승객운송
71544,id_071545,NaN,NaN,NaN,"세라믹, 금속","재료, 조립",NaN
28800,id_028801,NaN,NaN,NaN,사업장에서,고객의요구에따라,자동차부품및 내장품소매


In [ ]:
pd.DataFrame(data=train.isnull().sum()/len(train), columns=['nan_ratio'])

,nan_ratio
AI_id,0.000000
digit_1,0.000000
digit_2,0.000000
digit_3,0.000000
text_obj,0.016677
text_mthd,0.043619
text_deal,0.067652


In [ ]:
pd.DataFrame(data=test.isnull().sum()/len(test), columns=['nan_ratio'])

,nan_ratio
AI_id,0.00000
digit_1,1.00000
digit_2,1.00000
digit_3,1.00000
text_obj,0.01811
text_mthd,0.02968
text_deal,0.06161


In [ ]:
train.dtypes

AI_id        object
digit_1      object
digit_2       int64
digit_3       int64
text_obj     object
text_mthd    object
text_deal    object
dtype: object

In [ ]:
test.dtypes

AI_id         object
digit_1      float64
digit_2      float64
digit_3      float64
text_obj      object
text_mthd     object
text_deal     object
dtype: object

In [ ]:
# A, B, D가 상대적으로 매우 부족
train_count = train.groupby(by=["digit_1"]).count()
train_count["AI_id"]

digit_1
A      1064
B       424
C    105192
D       756
E      2255
F     35050
G    246472
H     98038
I    187425
J     10862
K     10378
L     40140
M     28434
N     17701
O      2965
P     46610
Q     36087
R     29751
S    100396
Name: AI_id, dtype: int64

In [ ]:
train.groupby(by=["digit_2"]).count()["AI_id"]

digit_2
1       827
2       121
3       116
5         1
6         7
      ...  
90     4504
91    25247
94    23625
95    20561
96    56210
Name: AI_id, Length: 74, dtype: int64

In [ ]:
train.groupby(by=["digit_3"]).count()["AI_id"]

digit_3
11       396
12       254
14       177
20       121
31        13
       ...  
951     1508
952    12142
953     6911
961    43040
969    13170
Name: AI_id, Length: 225, dtype: int64

In [ ]:
# digit은 classification을 위한 것이므로 연속형 변수 dtype인 int, float를 범주형 변수를 위한 dtype인 str으로 변경

train.loc[:,'digit_2'] = train.loc[:,'digit_2'].astype(str)
train.loc[:,'digit_3'] = train.loc[:,'digit_3'].astype(str)
test.loc[:,'digit_1'] = test.loc[:,'digit_1'].astype(str)
test.loc[:,'digit_2'] = test.loc[:,'digit_2'].astype(str)
test.loc[:,'digit_3'] = test.loc[:,'digit_3'].astype(str)

In [ ]:
train.dtypes

AI_id        object
digit_1      object
digit_2      object
digit_3      object
text_obj     object
text_mthd    object
text_deal    object
dtype: object

In [ ]:
test.dtypes

AI_id        object
digit_1      object
digit_2      object
digit_3      object
text_obj     object
text_mthd    object
text_deal    object
dtype: object

In [ ]:
len(train)

1000000

# Data
### input
* text_obj: 사업 대상, 무엇을 가지고/원재료, 영업장소 등
* text_mthd: 사업 방법, 어떤 방법으로/주요 영업, 생산활동
* text_deal: 사업 취급품목, 생산, 제공하였는가/최종 재화, 용역

### output
* digit_1: 대분류 코드
* digit_2: 중분류 코드 (데이터는 1 ~ 8인데 산업 분류표에는 01 ~ 08로 되어있음)
* digit_3: 소분류 코드 (중분류 코드 + n)
* 소분류 코드만 있으면 대, 중분류 판별 가능

### Cases
1. 
  * input에서 각각 column의 의미가 분명하지 않다고 가정하고 하나의 text로 만들기
  * output에서 코드를 합쳐서(대분류+소분류) 하나의 코드를 예측하기 -> submission에서 다시 대중소분류 나누기
  * 모델 하나로 해결되어 간단함.
  * 각 변수의 의미가 반영되지 않음
  * input 모두가 결측치인 경우가 아니면 행이 제거되지 않음

# Case 1. one-to-one classification (one text one label)

In [ ]:
# NaN은 덧셈이 불가능하므로 빈 문자열로 교체

train = train.fillna('')
test = test.fillna('')

In [ ]:
# 변수 합치기

train.loc[:,"label"] = train.loc[:,"digit_1"] + train.loc[:,"digit_3"]
train.loc[:,"text"] = train.loc[:,"text_obj"] + train.loc[:,"text_mthd"] + train.loc[:,"text_deal"]
test.loc[:,"text"] = test.loc[:,"text_obj"] + test.loc[:,"text_mthd"] + test.loc[:,"text_deal"]

In [ ]:
columns = ["AI_id", "text", "label"]
train = train.loc[:,columns]
train.sample(5)

,AI_id,text,label
933871,id_0933872,부동산에서계약및중개를통해부동산중개서비스,L682
431390,id_0431391,철절삭가공각종부품,C259
383637,id_0383638,보험회사에서사고 및재산손해에 대해보험업무를 수행,K651
223802,id_0223803,중학교중학교 교육을 통해고등학교 진학을 위한 교육 서비스,P852
417613,id_0417614,음식점에서접객시설을갖추고한식제공,I561


In [ ]:
train_count = train.groupby(by=["label"]).count()
train_count["AI_id"]

label
A11       396
A12       254
A14       177
A20       121
A31        13
        ...  
S951     1508
S952    12142
S953     6911
S961    43040
S969    13170
Name: AI_id, Length: 225, dtype: int64

In [ ]:
train['text'] = train['text'].str.replace("[^a-zA-Z가-힣]","", regex=True)

In [ ]:
train['text'] = train['text'].str.replace('^ +', "", regex=True)

In [ ]:
# text는 합친 것이므로 text_obj, text_mthd, text_deal 중 하나라도 있으면 사용 가능
# text_obj, text_mthd, text_deal가 모두 빈 문자열인 경우는 train dataset에 없었다.

train[train["text"]==""]

,AI_id,text,label


In [ ]:
len(train)

1000000

In [ ]:
# TODO : label 정리하고 label encoder 만들기
len(train["label"].unique())

224

In [ ]:
 sorted(train["label"].unique())

['A11',
 'A12',
 'A14',
 'A20',
 'A31',
 'A32',
 'B51',
 'B61',
 'B62',
 'B71',
 'B72',
 'B80',
 'C101',
 'C102',
 'C103',
 'C104',
 'C105',
 'C106',
 'C107',
 'C108',
 'C111',
 'C112',
 'C120',
 'C131',
 'C132',
 'C133',
 'C134',
 'C139',
 'C141',
 'C142',
 'C143',
 'C144',
 'C151',
 'C152',
 'C161',
 'C162',
 'C163',
 'C171',
 'C172',
 'C179',
 'C181',
 'C182',
 'C191',
 'C192',
 'C201',
 'C202',
 'C203',
 'C204',
 'C205',
 'C211',
 'C212',
 'C213',
 'C221',
 'C222',
 'C231',
 'C232',
 'C233',
 'C239',
 'C241',
 'C242',
 'C243',
 'C251',
 'C252',
 'C259',
 'C261',
 'C262',
 'C263',
 'C264',
 'C265',
 'C266',
 'C271',
 'C272',
 'C273',
 'C274',
 'C281',
 'C282',
 'C283',
 'C284',
 'C285',
 'C289',
 'C291',
 'C292',
 'C301',
 'C302',
 'C303',
 'C304',
 'C311',
 'C312',
 'C313',
 'C319',
 'C320',
 'C331',
 'C332',
 'C333',
 'C334',
 'C339',
 'C340',
 'D351',
 'D352',
 'D353',
 'E360',
 'E370',
 'E381',
 'E382',
 'E383',
 'E390',
 'F411',
 'F412',
 'F421',
 'F422',
 'F423',
 'F424',
 'F4

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# encoder = LabelEncoder()
# encoder.fit(train["label"])
# X_train_encoded = encoder.transform(train["label"])
# X_train_encoded

# for label in np.unique(test["label"]):
#     if label not in encoder.classes_:
#         encoder.classes_ = np.append(encoder.classes_, label) 
# X_test_encoded = encoder.transform(test["label"])

array([220, 125, 122, ..., 127, 204, 146])

In [ ]:
# class IndustryDataset(torch.utils.data.Dataset):
#   def __init__(self, dataset, is_train=True):
#     self.dataset = dataset
#     self.text = self.dataset["text"]
#     self.is_train = is_train
#     if is_train:
#       self.labels = self.dataset["label"]

#   def __getitem__(self, idx):
#     text = self.text[idx]
#     item = tokenizer(
#         text,
#         max_length = 144,
#         padding = "max_length",
#         truncation=True,
#         return_tensors = "pt",
#         add_special_tokens=True,
#         return_token_type_ids=False
#         )
#     if self.is_train:
#       labels = self.labels[idx]
#       item['labels'] = torch.tensor(labels)
#     item["input_ids"] = item["input_ids"].squeeze(0)
#     item["attention_mask"] = item["attention_mask"].squeeze(0)
#     return item

#   def __len__(self):
#     return len(self.dataset)